<a href="https://colab.research.google.com/github/tuananhnguyen512/AI2023_class/blob/main/ai_w6_24_3_23.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install scikit-fuzzy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 994.0/994.0 KB 31.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-fuzzy: filename=scikit_fuzzy-0.4.2-py3-none-any.whl size=894086 sha256=d79d78069b1d02f334c8648c3610f44f478d653135bbf527e405213d6ae806c7
  Stored in directory: /root/.cache/pip/wheels/32/2c/a1/a90a7d7dd8448ec029f298a61f3490275e99b17aa348be675c
Successfully built scikit-fuzzy


In [ ]:

import numpy as np
import skfuzzy as fuzz 
from skfuzzy import control as ctrl 

#DC motor control. 
sp = ctrl.Antecedent(np.arange(17, 29, 0.5), 'sp')
cv = ctrl.Antecedent(np.arange(0, 41, 0.5), 'cv')
mv = ctrl.Consequent(np.arange(0, 260, 0.5), 'mv')

# range: 18 - 27
sp['very_low'] = fuzz.trimf(sp.universe, [18,18,20])
sp['low'] = fuzz.trimf(sp.universe, [18,20,22])
sp['normal'] = fuzz.trimf(sp.universe, [20,22,24])
sp['high'] = fuzz.trimf(sp.universe, [22,24,27])
sp['very_high'] = fuzz.trimf(sp.universe, [24,27,27])

#range: 0 - 40
cv['very_low'] = fuzz.trimf(cv.universe, [0,0,10])
cv['low'] = fuzz.trimf(cv.universe, [0,10,15])
cv['normal'] = fuzz.trimf(cv.universe, [10,15,25])
cv['high'] = fuzz.trimf(cv.universe, [15,25,40])
cv['very_high'] = fuzz.trimf(cv.universe, [25,40,40])

#range: 0 - 60 Hz
# mv['very_low'] = fuzz.trimf(mv.universe, [0,0,10])
# mv['low'] = fuzz.trimf(mv.universe, [0,10,20])
# mv['normal'] = fuzz.trimf(mv.universe, [10,20,40])
# mv['high'] = fuzz.trimf(mv.universe, [20,40,60])
# mv['very_high'] =  fuzz.trimf(mv.universe, [40,60,60])

# OR range: 0 - 255 (pwm)
mv['very_low'] = fuzz.trimf(mv.universe, [0,0,70])
mv['low'] = fuzz.trimf(mv.universe, [0,60,120])
mv['normal'] = fuzz.trimf(mv.universe, [70,135,200])
mv['very_high'] =  fuzz.trimf(mv.universe, [200,255,255])

#Rule
rule1 = ctrl.Rule(sp['very_low']&cv['very_low']
                    | sp['very_low']&cv['low']
                    | sp['low']&cv['very_low'] ,mv['very_low'])

rule2 = ctrl.Rule(sp['low']&cv['normal']
                    | sp['very_low']&cv['normal']
                    | sp['low']&cv['low'] 
                    | sp['normal']&cv['low']
                    | sp['normal']&cv['very_low'],mv['low'])

rule3 = ctrl.Rule(sp['very_low']&cv['high']
                    | sp['low']&cv['high']
                    | sp['normal']&cv['normal'] 
                    | sp['high']&cv['low']
                    | sp['high']&cv['very_low'] ,mv['normal'])

rule4 = ctrl.Rule(sp['very_high']&cv['very_low']
                    | sp['very_high']&cv['low']
                    | sp['very_high']&cv['normal']
                    | sp['high']&cv['normal'] 
                    | sp['normal']&cv['high']
                    | sp['low']&cv['very_high']
                    | sp['normal']&cv['very_high'] 
                    | sp['very_low']&cv['very_high'] ,mv['high'])

rule5 = ctrl.Rule(sp['high']&cv['high'] 
                    | sp['very_high']&cv['high']
                    | sp['high']&cv['very_high'] 
                    | sp['very_high']&cv['very_high'],mv['very_high'])


mv_ctrl = ctrl.ControlSystem([rule1,rule2,rule3,rule4,rule5])
mving = ctrl.ControlSystemSimulation(mv_ctrl)

mving.input['sp'] = 25
mving.input['cv'] = 33
mving.compute()

print(mving.output['mv'])
mv.view(sim=mving)

ValueError: ignored